# Data Extraction, Loading & Concatenation

## Load Libraries

In [4]:
import pandas as pd
import os

## Function to Load and Concatenate data files

In [5]:
def load_and_concatenate_files(file_path, chunk_size=10**4, separator='$'):
    """
    Loads and concatenates data from a list of file paths with error handling.
    
    Parameters:
    file_path (str): File path to read.
    chunk_size (int): Number of rows per chunk to read at a time.
    separator (str): Delimiter for the CSV file.
    
    Returns:
    pd.DataFrame: A concatenated DataFrame from all the file chunks if successful, None otherwise.
    """
    chunks = []

    try:
        with open(file_path, 'r', encoding='ASCII', errors='replace') as file:
            for chunk in pd.read_csv(file, sep=separator, chunksize=chunk_size, on_bad_lines='skip'):
                chunks.append(chunk)
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
        return None

    if chunks:
        try:
            concatenated_df = pd.concat(chunks, ignore_index=True)
            concatenated_df.columns = concatenated_df.columns.str.lower()
            return concatenated_df
        except Exception as e:
            print(f"Error concatenating data: {e}")
            return None
    else:
        print("No data to concatenate.")
        return None


## Construct the year by quarter FAERS data files

In [ ]:
'''
For loop:
This code script iterates through the FAERS data files from years 2013 to 2023

Load and concatenate:
It applies the load_and_concatenate function, and merges all the data files into a final_df.

Create and save the final dataframe:
This is the final data extraction step before any data cleaning or anaylsis is performed.
'''


# Initialize an empty list to store dataframes
all_data = []

# Loop through the years 2013 to 2023
for year in range(2013, 2023):
    # Loop through the quarters Q1 to Q4
    for quarter in ['Q1', 'Q2', 'Q3', 'Q4']:
        
        # Construct the file names
        drugs_file = f'../drugs files/faers_ascii_{year}{quarter}/ASCII/DRUG{str(year)[-2:]}{quarter}.txt'
        indications_file = f'../drugs files/faers_ascii_{year}{quarter}/ASCII/INDI{str(year)[-2:]}{quarter}.txt'
        reactions_file = f'../drugs files/faers_ascii_{year}{quarter}/ASCII/REAC{str(year)[-2:]}{quarter}.txt'
        reportsource_file = f'../drugs files/faers_ascii_{year}{quarter}/ASCII/RPSR{str(year)[-2:]}{quarter}.txt'
        demo_file = f'../drugs files/faers_ascii_{year}{quarter}/ASCII/DEMO{str(year)[-2:]}{quarter}.txt'


        # Load and concatenate all the FAERS data files
        reactions = load_and_concatenate_files(reactions_file)
        reactions.drop(columns=['caseid'], inplace=True)

        indications = load_and_concatenate_files(indications_file)
        indications.drop(columns=['caseid'], inplace=True)

        drugs = load_and_concatenate_files(drugs_file)
        drugs['drugname'] = drugs['drugname'].fillna('') # Fill NaN values in 'drugname' with an empty string
        drugs = drugs[drugs['drugname'].str.contains('VENTOLIN')] # Filter by Ventolin drugname
        drugs.drop(columns=['caseid'], inplace=True) # drop the caseid column

        reportsource = load_and_concatenate_files(reportsource_file)
        reportsource.drop(columns=['caseid'], inplace=True) # drop the caseid column

        demographics = load_and_concatenate_files(demo_file)
        demographics.drop(columns=['caseid'], inplace=True) # drop the caseid column

        # Merged files
        drugs_react = pd.merge(drugs,reactions, on = 'primaryid', how = 'inner')
        drugs_reac_indi = pd.merge(drugs_react,indications, on = 'primaryid', how = 'inner')
        drugs_reac_indi_demo = pd.merge(drugs_reac_indi,demographics, on = 'primaryid', how = 'inner')
        drugs_reac_indi_demo_source = pd.merge(drugs_reac_indi_demo,reportsource, on = 'primaryid', how = 'inner')
        
        # Final_df is comprised of the merged dataframes of the drugs, reactions, indications, demogrphics, and report source code.
        final_df = drugs_reac_indi_demo_source.copy()

        try:
            # Create the directory if it doesn't exist
            output_dir = f'../final dataset/Drugs/{year}'
            os.makedirs(output_dir, exist_ok=True)

            # Save the CSV file - Filtered by Ventolin
            output_file = os.path.join(output_dir, f'Ventolin_df_{year}{quarter}.csv')
            
            # Save the CSV file - Unfiltered by any drug (requried if analysis of non ventolin drugs vs ventolin drugs is pursued)
            # output_file = os.path.join(output_dir, f'Drugs_df_{year}{quarter}.csv')
            
            # final df saved to output directory with output file name
            final_df.to_csv(output_file, index=False)

            print(f"File saved: {output_file}")
    

        except FileNotFoundError:
                    print(f"File not found") # catch any errors if output has failed.


File saved: ../final dataset/Drugs/2024/Ventolin_df_2024Q1.csv
File saved: ../final dataset/Drugs/2024/Ventolin_df_2024Q2.csv


## Load Ventolin data - Construct final dataframe

In [ ]:
# Initialize an empty list to store dataframes
all_data = []

# Loop through the years 2013 to 2023
for year in range(2013, 2023):
    # Loop through the quarters Q1 to Q4
    for quarter in ['Q1', 'Q2', 'Q3', 'Q4']:
        # Construct the file name
        file_name = f"../final dataset/Drugs/{year}/Ventolin_df_{year}{quarter}.csv"
        
        try:
            # Read the CSV file
            df = pd.read_csv(file_name)
            # Optionally, add a column to track the year and quarter
            df['year_quarter'] = f'{year}_{quarter}'
            df['year'] = year
            df['quarter'] = quarter
            # Append the dataframe to the list
            all_data.append(df)
        except FileNotFoundError:
            print(f"File not found: {file_name}")

# Combine all dataframes into a single dataframe
final_df = pd.concat(all_data, ignore_index=False)



File not found: ../final dataset/Drugs/2024/Ventolin_df_2024Q3.csv
File not found: ../final dataset/Drugs/2024/Ventolin_df_2024Q4.csv


## Save Dataframe

In [ ]:
# Save the final dataframe
final_df.to_csv('../final dataset/Final_df.csv')